In [29]:
import json
from pathlib import Path
from operator import itemgetter
import itertools

home_in_path = Path('/mnt/data/factcheck/twitter')
source_path = Path(home_in_path, 'people/text/tweets_with_titles.jsonl')
jd_annotated_path = Path(home_in_path, 'OLD/tweet_annotations.jsonl')
configs_path = Path(home_in_path, 'OLD/tweet_label_config.json')

out_path = Path('/mnt/data/factcheck/tweets/classify/data')

def load_jsonl(path):
    with open(path) as f:
        return [json.loads(line) for line in f]

## Get label configs

In [5]:
with open(configs_path) as f:
    label_inp = json.load(f)
label_configs = {item['id']: item['text'] for item in label_inp}
print(label_configs)

{1: 'negative', 2: 'positive', 4: 'claim', 5: 'insult', 6: 'unusable', 7: 'quote'}


## Analyse the tweets_with_titles.jsonl file

In [6]:
twt = load_jsonl(source_path)

In [7]:
print(len(twt))
print(twt[0])

185161
{'id': 1422203548138450949, 'text': 'Dnes je poslední den pro podání žádosti o nový kompenzační bonus. Dosud vyplatila živnostníkům a dohodářům pomoc za více než 14 miliard korun 👍'}


## Analyse the file with annotations by JD

In [8]:
anno = load_jsonl(jd_annotated_path)

In [9]:
print(jd_annotated_path)
print(len(anno))
print(anno[0])

/mnt/data/factcheck/twitter/OLD/tweet_annotations.jsonl
1000
{'id': 13, 'text': 'Vy se klidne modlete k te sve uhlikove Grete ale ceskou energetiku nechte jinym. Bude to tak lepsi', 'annotations': [{'label': 7, 'user': 1, 'created_at': '2021-03-15T14:56:25.587860Z', 'updated_at': '2021-03-15T14:56:25.587892Z'}, {'label': 5, 'user': 1, 'created_at': '2021-03-15T14:56:27.748825Z', 'updated_at': '2021-03-15T14:56:27.748861Z'}], 'meta': {'id': 1191099971174588419, 'username': 'dolejsjiri', 'original': '@vitekmilan @Tajda_cz @GreenpeaceCzech @czKSCM Vy se klidne modlete k te sve uhlikove Grete ale ceskou energetiku nechte jinym. Bude to tak lepsi  https://t.co/beBTtPu2uB'}, 'annotation_approver': None, 'comment_count': 0}


### split according to config classes

In [10]:
for num, text in label_configs.items():
    sub = [item for item in anno if item['annotations'][0]['label']==num]
    print(num, text, len(sub))
    texts = [item['text'] for item in sub[:5]]
    print(*texts, sep='\n')
    print()
    
quote = [item for item in anno if item['annotations'][0]['label']==7]
print(*quote[:2], sep='\n')

1 negative 1
Danit odpovědnost je složitě ospravedlnitelné i v době vážné krize, v období růstu je to zlodějna.   [Babiš hledá peníze na své dárky. Z pojišťoven chce odčerpat deset miliard - Echo24.cz]

2 positive 0


4 claim 174
Senát právě přijal zvláštní usnesení k situaci v Číně: hnutí Falun Gong, situace křesťanů, muslimů a dalších náboženských a etnických menšin musí být součástí jednání ústavních činitelů 🇨🇿 s představiteli Čínské lidové republiky.
Obdivuji lidi, kteří kategoricky komentují text, který nikdy nečetli. V Istanbulské smlouvě se o muslimech nepíše. Zmátlo Vás místo podpisu.
Při práci poslanců je asi pod největším drobnohledem jejich účast na hlasováních. Od té se totiž odvíjejí další aktivity jako schvalování zákonů či podoba diskusí během schůzí. Nejaktivnější jsou v tomto ohledu Piráti. 🏴 [První rok sněmovny: na hlasování chodí nejvíce Piráti, sedm poslanců ANO dosud nepromluvilo — ČT24 — Česká televize]
Koukal jsem, že těch 150 fleků pro komunisty ve státních fir

### conclusion
It seems that 'unusable' does not give sufficient context and 'quote' is some sort of proclamation or opinion (without being insulting).

As we seem to be mostly interested in tweets with dis/provable claims, I will merge classes 5-7 into one.

## JD vs JP annotation of same tweets: comparison of labels

In [26]:
path = Path('/mnt/data/factcheck/tweets/classify/data/random_anno.jsonl')
out_path = Path('/mnt/data/factcheck/tweets/classify/data/jd_jp_anno.jsonl')
f = open(out_path, "wt")

my_anno = load_jsonl(path) # id, text, label
jd_anno_orig = load_jsonl(jd_annotated_path) # id, text, annotations[0][label]
jd_anno_edit = [{'id': item['id'], 'metaid': item['meta']['id'], 'text': item['text'],
                'label': 1 if item['annotations'][0]['label']==4 else 0}
                 for item in jd_anno_orig]

jd_id_to_label =  {item['id']: item['label'] for item in jd_anno_edit}
jd_id_to_metaid = {item['id']: item['metaid'] for item in jd_anno_edit}

same = 0
for item in my_anno:
    my_id, my_text, my_label = itemgetter('id', 'text', 'label')(item)
    jd_label = jd_id_to_label[my_id]
    meta_id = jd_id_to_metaid[my_id]
    if my_label == jd_label:
        same += 1
    else:
        print('jp_label:', my_label, 'jd_label:', jd_label, my_text)
    outdict = {'id': meta_id, 'text': my_text, 'label': jd_label, 'label_jp': my_label}
    print(outdict, file=f)

print('\n', 'same annotations: ', same, '  annotations total: ', len(my_anno), sep='')
f.close()

jp_label: 1 jd_label: 0 Architekta Eva Jiřičná dnes slaví své 80. narozeniny
jp_label: 0 jd_label: 1 Diskuse pana prezidenta se studenty dvou gymnázií v Českém Těšíně s českým i polským vyučovacím jazykem.
jp_label: 0 jd_label: 1 Sháním stážisty! Slíbil jsem, že se pokusím politiku otevírat mladým lidem. Tenhle slib plním moc rád. Bylo by podle mě fajn nechat je i nakouknout pod pokličku a osahat si to. Tak kdybyste o někom ze střední či vysoké věděli, tak více informací tady:  [Facebook]
jp_label: 0 jd_label: 1 Varovali jsme, že počínání pana premiéra je v rozporu s evropským právem. Pan Babiš tomu celou dobu mohl zabránit. Stačilo dát si věci do pořádku.  [Evropská komise nebude vyplácet dotace pouze Agrofertu. Čeká na vyřešení kauzy Babiš - Aktuálně.cz]
jp_label: 1 jd_label: 0 Český patent fi je revolucí ve stavebnictví. Z unikátního materiálu už vzniká první dům❗️  [Pozitivní zprávy] via cc
jp_label: 0 jd_label: 1 TGM říkal, že rozčilení není program. V soudobé Přítomnosti se už al

## Annotate

- issue: expression with [] containing news message
- filter out tweets that are replies (do not have the reply list empty)
- what about quotes, e.g. Ferjenčík: 
-  or Nový ředitel hotelu Vladimír Novák mě ujistil, že bude s potomky rodiny architektů komunikovat a celá rekonstrukce by měla probíhat citlivě.
- tendency to focus on claims including number, tendency not to include factual claims that are not verifiable with respect to the fact base (V neděli budu v  komentovat aktuální politickou situaci. Těším se. Ve studiu se mnou budou ministryně pro místní rozvoj Klára Dostálová a místopředseda SPD Radim Fiala.)

In [68]:
from operator import itemgetter
import textwrap
import random

labelled_path = Path(out_path, 'labelled.jsonl')

batch_size = 300
subset = random.sample(twt, batch_size)

label_map = {'':0, ' ':1, 'f':11, 'g':10}
# 0==clearly not a statement, 1==clearly a statement
# 10==probably not a statement, 11==probably a statement
# random01 and random02 do not use 8 and 9 labels

for workcount, item in enumerate(subset, 1):
    with open(labelled_path, 'at') as f:
        id_, text = itemgetter('id', 'text')(item)
        wrap_lines = textwrap.wrap(text, width=80)
        prompt = f'[{workcount}/{batch_size}] ' + '\n'.join(wrap_lines) + '\n'
        i = input(prompt)
        while i not in ' fgc':  # ('' in 'afgc')==True
            i = input(prompt)
        if i == 'c':
            break
        label = label_map[i]
        json_dict = {
            'id': id_, 'text': text, 'label': label
        }
        string = json.dumps(json_dict, ensure_ascii=False)
        print(string, file=f)

[1/300]  Chtělo ny to něco v Izraelském stylu.
 c


## Joining annotations

In [31]:
export_path_unfiltered = Path(out_path, 'unfiltered.jsonl')
export_path_filtered   = Path(out_path, 'all_lines.jsonl')

def save_jsonl(export_path, data):
    with open(export_path, 'wt') as f:
        for l in data:
            print(json.dumps(l, ensure_ascii=False), file=f)

jd = load_jsonl(jd_annotated_path)
jd_edit = [{'id': item['meta']['id'], 'text': item['text'], 'label': item['annotations'][0]['label']}
            for item in jd]
for item in jd_edit:
    item['label'] = 1 if item['label']==4 else 0

files = [open(out_path / f'random{no:02d}.jsonl') 
         for no in range(2,11)]
all_lines = [json.loads(l) for l in itertools.chain(*files)]  # id, text, label

all_lines.extend(jd_edit)

save_jsonl(export_path_unfiltered, all_lines)

# filter out lines where I was not sure while annotating
all_lines = [line for line in all_lines if line['label'] not in {10, 11}]

import re
reg = re.compile(r'\[.*\]')
all_lines = [line for line in all_lines if reg.search(line['text']) is None]

save_jsonl(export_path_filtered, all_lines)

print('before truncation', len(all_lines))
all_lines = all_lines[:2500]
print('after truncation ', len(all_lines))

before truncation 2531
after truncation  2500


#### saving joined annotations

In [78]:
test = load_jsonl(export_path)
print(len(test))

2500
